In [1]:
import gmaps
import numpy as np
import xml.etree.ElementTree as ET
from datetime import datetime,timedelta
from datetime import tzinfo
import dateutil.parser

In [2]:
api_key = None
with open("key.txt") as f:
    api_key = f.readline()
    f.close()
if api_key:
    gmaps.configure(api_key=api_key)
    print("API Configed")

API Configed


In [3]:
import numpy as np
import xml.etree.ElementTree as ET
from datetime import datetime,timedelta
from datetime import tzinfo


In [99]:
def getTrkSeg(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    track = root.find("{http://www.topografix.com/GPX/1/1}trk")
    trkseg = track.find('{http://www.topografix.com/GPX/1/1}trkseg')
    return trkseg
def trkSeg2np(trkseg):
    segs = len(trkseg)
    data = np.zeros([segs,4])
    for i,child in enumerate(trkseg):
        data[i,0] = child.attrib['lat']
        data[i,1] = child.attrib['lon']
        elevElement = child[0] 
        timeElement = child[1]
        date = timeElement.text
        seconds,tz = convert2number(date)
        data[i,2] = elevElement.text
        data[i,3] = seconds
    return data
def convert2number(time):
    time2 = dateutil.parser.parse(time)
    tz = time2.tzinfo
    time_temp = datetime(1970,1,1,tzinfo=tz)
    delta1 = (time2-time_temp).total_seconds()
    total_seconds = (time2-time_temp).total_seconds()
    return total_seconds,tz
def convert2time(total_seconds,tz):
    delta = timedelta(seconds=total_seconds)
    time = delta+datetime(1970,1,1,tzinfo = tz)
    return time
def grab_outlier(data):
    grads = np.gradient(data,axis=0)
    grads[:,0] = grads[:,0] / grads[:,3]
    grads[:,1] = grads[:,1] / grads[:,3]
    dLats = np.abs(grads[:,0])
    dLons = np.abs(grads[:,1])
    dMan = dLons + dLats
    indexes = dMan > .0002
    # for i,d,grad in zip(indexes,dMan,grads):
    #     print(i,'%f' % d,'%f,%f,%f,%f' % (grad[0],grad[1],grad[2],grad[3]))
    return indexes
def make_features(data):
    locs = data[:,0:2]
    lines = []
    for p1,p2 in zip(locs[1:],locs[:-1]):
        lines += [gmaps.Line(p1,p2,stroke_weight=2.0,stroke_color='red')]
#         lines += [gmaps.Marker(p1)]
    return lines
def make_markers(data,goods):
    locs = data[:,0:2]
    marks = []
    color = []
    for l,g in zip(locs,goods):
        marks += [(l[0],l[1])]
        if g:
            color += ['red']
        else:
            color += ['blue']
    return marks,color
def fix_data(data,grad):
    data2 = data
    for i,g in enumerate(grad):
        if
    

In [105]:
trkseg = getTrkSeg("Ioane_run_4.gpx")
data = trkSeg2np(trkseg)
locs = grab_outlier(data)
start_lat,start_lon = data[0,0],data[0,1]
# for differnt layers
lines = make_features(data)
marks,colors = make_markers(data,locs)
# print(marks)

In [106]:
figure_layout = {'height':'800px'}
fig = gmaps.figure(layout=figure_layout,center=(start_lat,start_lon),zoom_level=13)
#Drawing Layer
drawing = gmaps.drawing_layer(features=lines)
fig.add_layer(drawing)
# Marking Layer
marking = gmaps.symbol_layer(marks,fill_color=colors,stroke_color=colors,fill_opacity=.5)
fig.add_layer(marking)
# 

In [107]:
fig

Figure(layout=FigureLayout(height='800px'))